In [ ]:
# !pip install opendatasets
# import opendatasets as od
# od.download("https://www.kaggle.com/competitions/cpe-ai-hackathon-2025-medical-image-classification")

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

In [ ]:
train = pd.read_csv("/content/cpe-ai-hackathon-2025-medical-image-classification/liver-fibrosis-severity-prediction/train.csv")
final = pd.read_csv("/content/cpe-ai-hackathon-2025-medical-image-classification/liver-fibrosis-severity-prediction/test.csv")
train = train.drop(columns=['subject'])
train = train.rename(columns={
    'image_name' : 'img',
    'SWE fibrosis stage' : 'stage',
    'TE result' : 'TE'
})

id_final = final.ID
final = final.drop(columns=['ID'])
final = final.rename(columns={
    'image_id' : 'img',
    'SWE fibrosis stage' : 'stage',
    'TE result' : 'TE'
})

In [ ]:
final.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 334 entries, 0 to 333
Data columns (total 4 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   img     334 non-null    object
 1   view    334 non-null    object
 2   stage   334 non-null    object
 3   TE      334 non-null    object
dtypes: object(4)
memory usage: 10.6+ KB


In [ ]:
train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1438 entries, 0 to 1437
Data columns (total 5 columns):
 #   Column   Non-Null Count  Dtype  
---  ------   --------------  -----  
 0   img      1438 non-null   object 
 1   view     1438 non-null   object 
 2   stage    1438 non-null   object 
 3   TE(kPa)  1438 non-null   float64
 4   TE       1438 non-null   object 
dtypes: float64(1), object(4)
memory usage: 56.3+ KB


In [ ]:
np.unique(train['view'])

array(['Intercostal', 'Liver/RK', 'Subcostal_hepatic_vein'], dtype=object)

In [ ]:
np.unique(train['stage'])

array(['-', 'F0-1', 'F2', 'F2 ', 'F3', 'F4'], dtype=object)

In [ ]:
np.unique(train['TE'])

array(['F0', 'F1', 'F2', 'F3', 'F4'], dtype=object)

In [ ]:
train = train.drop(columns='img')

In [ ]:
y_train = train['TE(kPa)']
x_train = train.drop(columns='TE(kPa)')

In [ ]:
x_train

,view,stage,TE
0,Intercostal,-,F2
1,Liver/RK,F0-1,F0
2,Subcostal_hepatic_vein,F0-1,F0
3,Liver/RK,F0-1,F0
4,Intercostal,F0-1,F1
...,...,...,...
1433,Intercostal,F0-1,F0
1434,Liver/RK,F0-1,F0
1435,Liver/RK,F0-1,F1
1436,Intercostal,F0-1,F1


In [ ]:
final = final.drop(columns='img')
final

,view,stage,TE
0,Intercostal,-,F0
1,Subcostal_hepatic_vein,F2,F3
2,Intercostal,F2,F3
3,Liver/RK,F2,F3
4,Subcostal_hepatic_vein,F0-1,F0
...,...,...,...
329,Intercostal,F0-1,F4
330,Subcostal_hepatic_vein,F0-1,F4
331,Subcostal_hepatic_vein,F0-1,F0
332,Intercostal,F0-1,F0


In [ ]:
x_train['stage'] = x_train['stage'].str.strip().map({
    '-' : 0,
    'F0-1' : 1,
    'F2' : 2,
    'F3' : 3,
    'F4' : 4
})

x_train['TE'] = x_train['TE'].map({
    'F0' : 0,
    'F1' : 1,
    'F2' : 2,
    'F3' : 3,
    'F4' : 4
})

x_train = pd.get_dummies(x_train, columns=['view'], dtype=int)

final['stage'] = final['stage'].str.strip().map({
    '-' : 0,
    'F0-1' : 1,
    'F2' : 2,
    'F3' : 3,
    'F4' : 4
})

final['TE'] = final['TE'].map({
    'F0' : 0,
    'F1' : 1,
    'F2' : 2,
    'F3' : 3,
    'F4' : 4
})

final = pd.get_dummies(final, columns=['view'], dtype=int)

In [ ]:
x_train = x_train.values.astype(np.float32)
y_train = y_train.values.astype(np.float32).reshape(-1, 1)
tensor_x_train = torch.tensor(x_train)
tensor_y_train = torch.tensor(y_train)

final = final.values.astype(np.float32)
tensor_final = torch.tensor(final)

In [ ]:
class neural_network(nn.Module):
    def __init__(self, input_dim, hidden_dims=[256, 128, 64, 32], dropout_rate=0.3):
        super(neural_network, self).__init__()

        self.input_bn = nn.BatchNorm1d(input_dim)
        layers = []
        in_dim = input_dim

        for h_dim in hidden_dims:
            layers.append(Block(in_dim, h_dim, dropout_rate))
            in_dim = h_dim

        self.layers = nn.Sequential(*layers)
        self.output = nn.Linear(in_dim, 1)
        self.apply(self._init_weights)

    def forward(self, x):
        x = self.input_bn(x)
        x = self.layers(x)
        x = self.output(x)
        return x

    def _init_weights(self, m):
        if isinstance(m, nn.Linear):
            nn.init.kaiming_normal_(m.weight, nonlinearity='leaky_relu')
            if m.bias is not None:
                nn.init.constant_(m.bias, 0)

class Block(nn.Module):
    def __init__(self, in_dim, out_dim, dropout_rate):
        super(Block, self).__init__()
        self.fc = nn.Linear(in_dim, out_dim)
        self.bn = nn.BatchNorm1d(out_dim)
        self.act = nn.LeakyReLU(negative_slope=0.1)
        self.drop = nn.Dropout(dropout_rate)
        self.residual = (in_dim == out_dim)

    def forward(self, x):
        identity = x
        out = self.fc(x)
        out = self.bn(out)
        out = self.act(out)
        out = self.drop(out)
        if self.residual:
            out = out + identity * 0.5
        return out


In [ ]:
model = neural_network(x_train.shape[1])
error = nn.MSELoss()
optimizer = optim.Adam(model.parameters(), lr=0.01)
scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(optimizer, patience=5, factor=0.5)

epoch = 3000
for i in range(epoch):
    model.train()
    y_hat = model(tensor_x_train)
    loss = error(y_hat, tensor_y_train)

    optimizer.zero_grad()
    loss.backward()
    optimizer.step()

    if (i + 1) % 10 == 0:
        print(f"Epoch [{i+1}/{epoch}] | Loss(MSE): {loss.item():.4f}")

Epoch [10/3000] | Loss(MSE): 21.2621
Epoch [20/3000] | Loss(MSE): 9.4891
Epoch [30/3000] | Loss(MSE): 6.5132
Epoch [40/3000] | Loss(MSE): 6.6678
Epoch [50/3000] | Loss(MSE): 5.8893
Epoch [60/3000] | Loss(MSE): 5.4632
Epoch [70/3000] | Loss(MSE): 5.9787
Epoch [80/3000] | Loss(MSE): 5.2920
Epoch [90/3000] | Loss(MSE): 5.1343
Epoch [100/3000] | Loss(MSE): 5.4049
Epoch [110/3000] | Loss(MSE): 4.7944
Epoch [120/3000] | Loss(MSE): 5.5572
Epoch [130/3000] | Loss(MSE): 5.2900
Epoch [140/3000] | Loss(MSE): 4.8570
Epoch [150/3000] | Loss(MSE): 5.0567
Epoch [160/3000] | Loss(MSE): 5.1593
Epoch [170/3000] | Loss(MSE): 4.7187
Epoch [180/3000] | Loss(MSE): 5.3839
Epoch [190/3000] | Loss(MSE): 5.0998
Epoch [200/3000] | Loss(MSE): 4.9794
Epoch [210/3000] | Loss(MSE): 4.7999
Epoch [220/3000] | Loss(MSE): 4.8208
Epoch [230/3000] | Loss(MSE): 4.4727
Epoch [240/3000] | Loss(MSE): 4.3294
Epoch [250/3000] | Loss(MSE): 4.7686
Epoch [260/3000] | Loss(MSE): 4.4819
Epoch [270/3000] | Loss(MSE): 4.6068
Epoch [28

In [ ]:
with torch.no_grad():
  y_hat = model(tensor_final)
y_hat = pd.DataFrame(y_hat)
summarize = pd.concat([id_final, y_hat], axis=1)
summarize.columns = ['ID', 'TE(kPa)']
summarize.to_csv('My_Answer.csv', index=False)
summarize

,ID,TE(kPa)
0,1,4.623334
1,2,8.907242
2,3,9.313368
3,4,8.869608
4,5,4.328938
...,...,...
329,330,22.027388
330,331,15.265818
331,332,4.354945
332,333,4.317220
